# 생활인구 데이터 홈페이지
https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do

In [1]:
from selenium import webdriver
import time
import folium
import os
import json
import pandas as pd
import numpy as np
import datetime

In [2]:
data_path = '/home/zimin/datasets/seoul/'
geo_path = data_path + 'skorea_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str['features'] = [i for i in geo_str['features'] if i['properties']['code'][:2] == '11']

In [3]:
pd.set_option('max_colwidth',500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [13]:
analysis = datetime.datetime(2019,11, 22)
week = {0 : '월요일', 1 : '화요일', 2 : '수요일', 3 : '목요일', 4 : '금요일', 5 : '토요일', 6 : '일요일'}
weekday = week[analysis.weekday()]
date = str(analysis.year) + str(analysis.month) + str(analysis.day) 

In [6]:
local = pd.read_csv(os.path.join(data_path, 'pre_local_PEOPLE_' + date + '.csv'), index_col=[2])

In [66]:
when = 21
who = ['140','240']
whose = who[0][1:] + '~' + str(eval(who[-1][1:] )+5)

In [67]:
date, weekday, when, whose

('20191122', '금요일', 21, '40~45')

In [68]:
check = local[(local['시간대구분'] == when)][['행정동이름','행정동코드'] + who]
check[who] = check[who].replace('*',0)
check[who] = check[who].astype(int)
check['sum'] = check[who].sum(axis=1)
check['행정동코드'] = check['행정동코드'].apply(str)

In [69]:
check.sort_values(by='sum',ascending=False).head(10)

,행정동이름,행정동코드,140,240,sum
기준일ID,,,,,
20191122,역삼1동,1123064,6363,3670,10033
20191122,여의동,1119054,4403,3141,7544
20191122,서교동,1114066,4009,3339,7348
20191122,서초3동,1122053,3174,2703,5877
20191122,종로1.2.3.4가동,1101061,3564,2073,5637
20191122,역삼2동,1123065,2683,2578,5261
20191122,진관동,1112071,2209,2982,5191
20191122,논현2동,1123053,2891,1879,4770
20191122,압구정동,1123077,2489,2222,4711


In [70]:
fn= date + weekday + ' ' +str(when)+ '시 ' + whose + '세' + '.html'

seoul_map = folium.Map(location =[37.5502, 126.982], zoom_start=11, tiles = 'cartodbpositron')
seoul_map.choropleth(geo_data = geo_str,
                    data = check,
                    columns = ['행정동코드', 'sum'],
                    fill_color = 'Reds',
                    bins = 9,
                    key_on = 'properties.code',
                    highlight = True,
                    fill_opacity = 0.7, line_opacity = 1,
                    legend_name = 'good_boy')

title_html = '''
             <h3 align="center" style="font-size:20px"><b>{}</b></h3>
             '''.format(fn)
seoul_map.get_root().html.add_child(folium.Element(title_html))

delay=1
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd() + '/html_data',mapfile=fn)
seoul_map.save('./html_data/' + fn)
browser = webdriver.Chrome('/usr/bin/chromedriver')
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('./screenshot/' + fn[:-5] + '.png')
browser.quit()